In [1]:
import numpy as np
import svox2
import torch
import kaolin


/home/tw554/plenoxels/svox2/utils.py:39: UserWarning: CUDA extension svox2.csrc could not be loaded! Operations will be slow.
Please do not import svox in the svox2 source directory.
  warn("CUDA extension svox2.csrc could not be loaded! " +


In [3]:
grid = svox2.SparseGrid(reso=[64,64,64],
                            center=0.,
                            radius=1.,
                            use_sphere_bound=False,
                            basis_dim=9,
                            use_z_order=True,
                            device='cuda',
                            basis_reso=32,
                            basis_type=svox2.__dict__['BASIS_TYPE_' + 'sh'.upper()],
                            backend='plane',
                            surface_init=None,
                            use_octree=False)

In [14]:
from PIL import Image
import torch
import torch.nn.functional as F
import numpy as np
from matplotlib import pyplot as plt
import ipywidgets as widgets
from termcolor import colored

def _normalized_grid(width, height, device='cuda'):
    """Returns grid[x,y] -> coordinates for a normalized window.
    
    Args:
        width, height (int): grid resolution
    """

    # These are normalized coordinates
    # i.e. equivalent to 2.0 * (fragCoord / iResolution.xy) - 1.0
    window_x = torch.linspace(-1, 1, steps=width, device=device) * (width / height)
    window_y = torch.linspace(1,- 1, steps=height, device=device)

    coord = torch.stack(torch.meshgrid(window_x, window_y)).permute(1,2,0)
    return coord


def look_at(camera_from, camera_to, width, height, mode='persp', fov=90.0, device='cuda'):
    """Vectorized look-at function, returns an array of ray origins and directions
    URL: https://www.scratchapixel.com/lessons/mathematics-physics-for-computer-graphics/lookat-function
    """

    camera_origin = torch.FloatTensor(camera_from).to(device)
    camera_view = F.normalize(torch.FloatTensor(camera_to).to(device) - camera_origin, dim=0)
    camera_right = F.normalize(torch.cross(camera_view, torch.FloatTensor([0,1,0]).to(device)), dim=0)
    camera_up = F.normalize(torch.cross(camera_right, camera_view), dim=0)

    coord = _normalized_grid(width, height, device=device)
    ray_origin = camera_right * coord[...,0,np.newaxis] * np.tan(np.radians(fov/2)) + \
                 camera_up * coord[...,1,np.newaxis] * np.tan(np.radians(fov/2)) + \
                 camera_origin + camera_view
    ray_origin = ray_origin.reshape(-1, 3)
    ray_offset = camera_view.unsqueeze(0).repeat(ray_origin.shape[0], 1)
    
    if mode == 'ortho': # Orthographic camera
        ray_dir = F.normalize(ray_offset, dim=-1)
    elif mode == 'persp': # Perspective camera
        ray_dir = F.normalize(ray_origin - camera_origin, dim=-1)
        ray_origin = camera_origin.repeat(ray_dir.shape[0], 1)
    else:
        raise ValueError('Invalid camera mode!')


    return ray_origin, ray_dir

ray_o, ray_d = look_at(camera_from=[-2.5,2.5,-2.5],
                       camera_to=[0,0,0],
                       width=128,
                       height=128,
                       mode='persp',
                       fov=30,
                       device='cuda')

In [5]:
feature_grid = grid.plane_data.T[None, ...].reshape(1, 4, 64, 64, 64)
mask = torch.ones([1, 64, 64, 64]) > 0.
# mask[0,0,0,0] = False
octrees, lengths, features = kaolin.ops.spc.feature_grids_to_spc(feature_grid, mask)
max_level, pyramids, exsum = kaolin.ops.spc.scan_octrees(octrees, lengths)
point_hierarchies = kaolin.ops.spc.generate_points(octrees, pyramids, exsum)

In [9]:
features.shape

torch.Size([262144, 4])

In [15]:
# ray_o = torch.tensor([[-1.1,-1.1,-1.1]]).cuda() * -1
# ray_d = torch.tensor([[1.,1.,1.]]).cuda() * -1
nugs_ridx, nugs_pidx, depth = kaolin.render.spc.unbatched_raytrace(octrees, point_hierarchies, pyramids[0], exsum, ray_o, ray_d, max_level)
nugs_pidx

tensor([262723, 262727, 262728,  ...,  74833,  74834,  74833], device='cuda:0',
       dtype=torch.int32)

In [23]:
nugs_ridx

tensor([    5,     5,     5,  ..., 16362, 16362, 16363], device='cuda:0',
       dtype=torch.int32)

In [26]:
nugs_pidx.shape

torch.Size([1300708])

In [25]:
torch.bincount(nugs_ridx).sum()

tensor(1300708, device='cuda:0')

In [28]:
kaolin.ops.spc.morton_to_points(nugs_pidx.long() - pyramids[0,1,max_level]).shape

torch.Size([1300708, 3])

In [125]:
nugs_pidx.shape

torch.Size([209])

In [93]:
nugs_pidx

tensor([262723, 262727, 262728,  ...,  74833,  74834,  74833], device='cuda:0',
       dtype=torch.int32)

In [94]:
ray_d[5]

tensor([ 0.8152, -0.3531,  0.4591], device='cuda:0')